In [21]:
import os
import cv2 as cv
import numpy as np
from joblib import Parallel, delayed

In [22]:
# Loading the images

IMG_FOLDER = "../data/images/train"
IMG_FILES = os.listdir(IMG_FOLDER)

In [23]:
# Getting illumination channels

def compute_patch(img, w, i, j):
    """
    Compute the minimum and maximum values of the patch centered at (i, j).

    Args:
        img (numpy.ndarray): The input image.
        w (int): Half the size of the patch.
        i (int): The row index of the center of the patch.
        j (int): The column index of the center of the patch.

    Returns:
        tuple: A tuple containing the minimum and maximum values of the patch.
    """
    width, height, _ = img.shape
    x1, x2 = max(0, i - w), min(width, i + w)
    y1, y2 = max(0, j - w), min(height, j + w)
    patch = img[x1:x2, y1:y2, :]
    return np.min(patch), np.max(patch)

def get_illumination_channels(img, w):
    """
    Compute the dark and bright channels of the input image.

    Args:
        img (numpy.ndarray): The input image.
        w (int): Half the size of the patch.

    Returns:
        tuple: A tuple containing the dark and bright channels.
    """
    width, height, _ = img.shape
    img_dark = np.zeros((width, height))
    img_bright = np.zeros((width, height))
    
    # Parallelize the loop
    results = Parallel(n_jobs=-1)(
        delayed(compute_patch)(img, w, i, j)
        for i in range(width)
        for j in range(height)
    )
    
    # Unpack results and assign to channels
    for idx, (min_val, max_val) in enumerate(results):
        i, j = divmod(idx, height)
        img_dark[i, j] = min_val
        img_bright[i, j] = max_val
    
    return img_dark, img_bright

In [24]:
# Getting the atmosphere values

def get_atmosphere(I, img_bright, p):
    """
    Estimates the atmospheric light value A from the input image and its bright channel.

    Args:
        I (numpy.ndarray): The input image.
        img_bright (numpy.ndarray): The bright channel of the input image.
        p (float): The percentage of the brightest pixels to consider.

    Returns:
        numpy.ndarray: The estimated atmospheric light value.
    """
    width, height = img_bright.shape

    # Flatten the image and bright channel
    flat_image = I.reshape(width * height, 3)
    flat_bright = img_bright.reshape(width * height)

    # Get indices of the top p% brightest pixels
    search_index = np.argsort(flat_bright)[::-1][:int(p * width * height)]
    brightest_pixels = flat_image[search_index]

    # Compute the mean of the brightest pixels to estimate atmospheric light
    A = np.mean(brightest_pixels, axis=0, dtype=np.float64)
    
    return A

In [36]:
# Getting the initial transmission map

def get_initial_transmission_map(A, bright_channel):
    """
    Compute the initial transmission map based on the estimated atmospheric light value and the bright channel.
    
    Args:
        A (numpy.ndarray): The estimated atmospheric light value.
        bright_channel (numpy.ndarray): The bright channel of the input image.
    
    Returns:
        numpy.ndarray: The normalized initial transmission map.
    """
    # Compute the maximum value of the estimated atmospheric light
    A_c = np.max(A)
    
    # Compute the initial transmission map
    init_t = (bright_channel - A_c) / (255 - A_c)
    
    # Normalize the initial transmission map to the range [0, 1]
    normalized_t = (init_t - np.min(init_t)) / (np.max(init_t) - np.min(init_t))
    
    return normalized_t

In [40]:
# Getting the refined transmission map

def get_refined_transmission_map(img, A, img_dark, img_bright, init_t, alpha, omega, w):
    img_n = np.empty(img.shape, dtype=img.dtype)
    
    for channel in range(0, 3):
        img_n[:, :, channel] = img[:, :, channel] / A[channel]
        
    img_dark_n, img_bright_n = get_illumination_channels(img_n, w)
    
    dark_t = 1 - omega * img_dark_n
    
    corrected_t = init_t
    
    diff_channel = img_bright - img_dark
    
    corrected_t = dark_t + init_t if diff_channel < alpha else corrected_t
    
    return np.abs(corrected_t)

In [26]:
img = cv.imread(os.path.join(IMG_FOLDER, IMG_FILES[0]))

print(img.shape)
print(img.dtype)

(4032, 3024, 3)
uint8


In [27]:
dark_channel, bright_channel = get_illumination_channels(img, 15)

print(dark_channel.shape, bright_channel.shape)

(4032, 3024) (4032, 3024)


In [39]:
print(dark_channel.dtype, bright_channel.dtype)
print(np.min(dark_channel), np.max(dark_channel))

atmosphere = get_atmosphere(img, bright_channel, 0.1)

print(atmosphere)

normalized = get_initial_transmission_map(atmosphere, bright_channel)

print(normalized.shape)
print(np.min(normalized), np.max(normalized))

float64 float64
0.0 33.0
[36.17546642 39.40819142 42.77321952]
(4032, 3024)
0.0 1.0


In [41]:
corrected_t = get_refined_transmission_map(img, atmosphere, dark_channel, bright_channel, normalized, 0.4, 0.75, 15)

print(corrected_t.shape)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()